# Лабораторная работа №2

## Подготока входных данных

In [37]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [39]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

In [40]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

In [43]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})
X

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
56094,0,6.0,25.4,0.0,0,115.0,0,0,44.0,41.0,56.0,31.0,1008.3,999.8,15.5,24.4,False,True
61516,1,12.0,19.1,0.2,1,46.0,1,1,19.0,28.0,55.0,56.0,1022.3,1021.9,15.9,17.6,False,False
46466,2,17.0,22.8,1.4,2,50.0,2,2,2.0,20.0,88.0,68.0,1016.0,1015.8,18.2,21.5,True,True
90743,3,18.9,26.6,3.4,3,52.0,3,3,30.0,31.0,75.0,64.0,1023.8,1021.1,23.5,24.9,True,True
18876,4,21.9,31.1,0.0,4,46.0,0,4,9.0,20.0,79.0,65.0,1011.8,1007.7,25.0,28.5,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108560,25,7.6,19.2,0.0,6,19.0,3,0,6.0,7.0,57.0,36.0,1027.4,1024.3,12.4,18.4,False,False
116428,24,20.7,30.0,0.0,5,30.0,8,8,7.0,17.0,51.0,51.0,1012.3,1011.2,26.1,27.4,False,False
117915,12,8.2,23.9,0.0,3,31.0,1,9,15.0,19.0,53.0,43.0,1025.3,1021.5,16.6,22.9,False,False
91201,3,9.0,21.6,0.0,13,41.0,9,9,19.0,20.0,40.0,31.0,1017.1,1013.7,16.0,20.1,False,False


In [45]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

In [47]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [53]:
from sklearn.svm import SVC

In [55]:
classifier = SVC()
params ={
    'C': [1,0.1,0.01,0.001],
    'gamma': [1,10,100,1000],
    'kernel':['linear','sigmoid','rbf']
}
grid = GridSearchCV(classifier, params, cv=2, n_jobs=-1,verbose=3)
grid.fit(X_train, y_train)
best_params = grid.best_params_

Fitting 2 folds for each of 48 candidates, totalling 96 fits


In [56]:
best_model = SVC(**best_params)
best_model.fit(X_train, y_train)
predicted = best_model.predict(X_test)
print('Used params:',best_params)
print('Evaluation:\n', metrics.classification_report(y_test, predicted,digits=5))

Used params: {'C': 1, 'gamma': 1, 'kernel': 'linear'}
Evaluation:
               precision    recall  f1-score   support

       False    0.86015   0.95713   0.90605      9003
        True    0.75799   0.46322   0.57503      2610

    accuracy                        0.84612     11613
   macro avg    0.80907   0.71017   0.74054     11613
weighted avg    0.83719   0.84612   0.83165     11613

